<a href="https://colab.research.google.com/github/jj132535/YOLOv8-human-video-detection/blob/main/YOLO_with_Vertical_Reframing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyYAML
!pip install ultralytics
!pip install roboflow

In [ ]:
import yaml
import ultralytics
import cv2
import numpy as np
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
#add your video path
video_path = '/content/dreamcatcher.mp4'

# !! PUT yolov8_trained_model.pt FILE INTO /content/ !!
trained_model_path = '/content/yolov8_trained_model.pt' #trained model

output_video_path = '/content/output_video_chuu.mp4'

In [ ]:
# 저장된 모델 불러오기
model = YOLO(trained_model_path)

In [ ]:
results = model.predict(source=video_path, stream=True)

In [ ]:

# 비디오 저장 설정 (세로 비율 9:16)
width, height = 1080, 1920
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))

# 초기 설정
frame_count = 0

# 각 프레임에 대해 탐지된 객체를 기준으로 리프레이밍
for r in results:
    frame = r.orig_img  # 원본 프레임 가져오기
    bboxes = r.boxes.xywh if hasattr(r, 'boxes') and r.boxes is not None else None
    frame_count += 1

    if frame_count >= 3:  # 첫 두 프레임은 건너뜀
        if bboxes is not None and bboxes.shape[0] > 0:
            # numpy 배열로 변환하여 사용
            bboxes = bboxes.cpu().numpy()

            # 중앙에 가장 가까운 객체 선택
            frame_center_x = frame.shape[1] / 2
            closest_bbox = min(bboxes, key=lambda bbox: abs(bbox[0] - frame_center_x))
            x_center, y_center, box_width, box_height = closest_bbox

            # 탐지된 객체의 중심을 기준으로 세로로 크롭
            x1 = int(x_center - width / 2)
            x2 = int(x_center + width / 2)
            y1 = 0
            y2 = frame.shape[0]  # 원래 높이를 유지

            # 프레임의 경계를 벗어나는 경우 조정
            x1 = max(0, x1)
            x2 = min(frame.shape[1], x2)

            cropped_frame = frame[y1:y2, x1:x2]  # 세로로 크롭된 프레임
            resized_frame = cv2.resize(cropped_frame, (width, height))

            # FPS를 원본 비디오와 동일하게 설정하여 속도 문제 해결
            out.write(resized_frame)  # 비디오에 작성
        else:
            print(f"No detections in frame {frame_count}")


In [ ]:
# VideoWriter 객체 닫기
out.release()

In [ ]:
print("Video reframing complete and saved to:", output_video_path)